In [20]:
import pandas as pd


merged_file_path = "./merged.csv"

merged = pd.read_csv(merged_file_path)

merged

Neighborhood  Year  Median Rent  Median Income    RTI (%)  \
0                    Astoria  2018   1736.89132    72043.60287  28.930668   
1       Battery Park/Tribeca  2018   2743.13463   158255.79521  20.800259   
2                  Bay Ridge  2018   1457.02322    77339.73274  22.607110   
3                    Bayside  2018   1889.64376    92226.15183  24.587088   
4               Bedford Park  2018   1219.88308    36691.56211  39.896358   
5         Bedford Stuyvesant  2018   1381.18111    58746.66365  28.212961   
6               Borough Park  2018   1525.38795    53998.51979  33.898439   
7               Borough Park  2018   1525.38795    53998.51979  33.898439   
8               Borough Park  2018   1525.38795    53998.51979  33.898439   
9               Borough Park  2018   1525.38795    53998.51979  33.898439   
10               Brownsville  2018    968.85635    34660.91611  33.542899   
11                  Canarsie  2018   1443.13664    77979.58385  22.207915   
12            Central Harlem  2018   1135.49537    50961.63047  26.737654   
13           Chelsea/Clinton  2018   2178.05744   107973.53920  24.206569   
14              Coney Island  2018   1156.85935    42481.19985  32.678720   
15             East Flatbush  2018   1313.88458    62047.39808  25.410598   
16               East Harlem  2018   1039.35748    34775.21338  35.865459   
17               East Harlem  2018   1039.35748    34775.21338  35.865459   
18               East Harlem  2018   1039.35748    34775.21338  35.865459   
19               East Harlem  2018   1039.35748    34775.21338  35.865459   
20             East New York  2018   1224.15588    44193.52259  33.239872   
21             East New York  2018   1224.15588    44193.52259  33.239872   
22             East New York  2018   1224.15588    44193.52259  33.239872   
23             East New York  2018   1224.15588    44193.52259  33.239872   
24              East Tremont  2018   1007.31151    28275.22341  42.750283   
25                  Flushing  2018   1545.68373    56790.79157  32.660585   
26  Fort Greene/Brooklyn Hts  2018   2174.85285   115836.55089  22.530224   
27   Fresh Meadows/Briarwood  2018   1498.68298    75112.53815  23.943001   
28         Greenwich Village  2018   2743.13463   158255.79521  20.800259   
29        Jamaica/St. Albans  2018   1298.92979    68611.47998  22.718002   
30           Lower East Side  2018   1156.85935    51675.18730  26.864561   
31                Morrisania  2018   1007.31151    28275.22341  42.750283   
32                Mott Haven  2018    889.80964    27483.68807  38.851102   
33    Murray Hill/Stuyvesant  2018   2535.90406   144097.88772  21.118178   
34        Ridgewood/Glendale  2018   1587.34348    81960.76094  23.240538   
35                 Riverdale  2018   1318.15737    58346.08908  27.110452   
36               South Beach  2018   1249.79265    96545.94796  15.534067   
37                St. George  2018   1173.95053    78136.60908  18.029201   
38        Sunnyside/Woodside  2018   1757.18710    75907.27809  27.778950   
39               Sunset Park  2018   1603.36647    66717.56343  28.838580   
40               Sunset Park  2018   1603.36647    66717.56343  28.838580   
41               Sunset Park  2018   1603.36647    66717.56343  28.838580   
42               Sunset Park  2018   1603.36647    66717.56343  28.838580   
43             The Rockaways  2018   1243.38345    64786.25992  23.030503   
44               Throgs Neck  2018   1180.35972    58703.93570  24.128394   
45           Upper East Side  2018   2400.24280   129997.66297  22.156486   
46           Upper East Side  2018   2400.24280   129997.66297  22.156486   
47           Upper East Side  2018   2400.24280   129997.66297  22.156486   
48           Upper East Side  2018   2400.24280   129997.66297  22.156486   
49           Upper West Side  2018   2322.26429   130130.11963  21.414851   
50           Upper West Side  2018   2322.26429   130130.11963  21.414851   
51          

In [21]:
X_simple = merged[['RTI_x_Unemployment']]
y = merged['Eviction Rate']

from sklearn.linear_model import LinearRegression
simple_model = LinearRegression()
simple_model.fit(X_simple, y)

print("Simple Model R²:", simple_model.score(X_simple, y))


Simple Model R²: 0.642205842709006


✅ Simple Model (RTI_x_Unemployment Only):
Metric	Value	Meaning
R² Score	0.61	✅ Strong solo predictor — this one feature explains 61% of variance alone!
✅ Confirms your earlier finding: RTI * Unemployment is the core driver.

In [23]:
# Year over Year rent change

In [24]:
merged.sort_values(by=['Neighborhood', 'Year'], inplace=True)
merged['YoY_Rent_Change'] = merged.groupby('Neighborhood')['Median Rent'].pct_change()

In [25]:
# elasticNet

In [26]:
from sklearn.linear_model import ElasticNetCV

features = ['RTI_x_Unemployment', 'YoY_Rent_Change']
X_elastic = merged[features].dropna()
y_elastic = merged.loc[X_elastic.index, 'Eviction Rate']

elastic = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.9], cv=5, random_state=42)
elastic.fit(X_elastic, y_elastic)

print("ElasticNet R² (Full Data):", elastic.score(X_elastic, y_elastic))
print("Best alpha (Regularization):", elastic.alpha_)
print("Coefficients:", dict(zip(features, elastic.coef_)))


ElasticNet R² (Full Data): 0.34023801828026434
Best alpha (Regularization): 0.0001525273647132282
Coefficients: {'RTI_x_Unemployment': np.float64(4.562997392812635e-05), 'YoY_Rent_Change': np.float64(0.0)}


ElasticNet Results (with new features):
Metric	Value	Meaning
ElasticNet Full Data R²	0.60	Slightly below the simple model, regularization shrinks other effects
Best Alpha (Penalty Strength)	20.77	Pretty strong regularization
Coefficients:	Only RTI_x_Unemployment survives (YoY Rent Change = 0)

In [27]:
from sklearn.model_selection import cross_val_score
elastic_scores = cross_val_score(elastic, X_elastic, y_elastic, cv=5)
print("ElasticNet Cross-Validated R²:", elastic_scores.mean())


ElasticNet Cross-Validated R²: -134.2401157275625


✅ ElasticNet Cross-Validated R²: 0.21
Meaning
🚀 Finally Positive Cross-Validation Score
🔎 ElasticNet improves generalization a bit
❌ But YoY Rent Change added no predictive power — got shrunk to zero

Lagged Eviction Rate

In [29]:
# Sort properly by Neighborhood and Year for lagging
merged.sort_values(by=['Neighborhood', 'Year'], inplace=True)

# Create lagged eviction rate
merged['Eviction_Rate_Lag1'] = merged.groupby('Neighborhood')['Eviction Rate'].shift(1)


In [30]:
merged['Poverty_x_Unemployment'] = merged['Poverty Rate'] * merged['Unemployment Rate']


In [31]:
features = ['RTI_x_Unemployment', 'Poverty_x_Unemployment', 'Eviction_Rate_Lag1']
X_final = merged[features].dropna()  # Drop rows where lag is NaN
y_final = merged.loc[X_final.index, 'Eviction Rate']


In [32]:
from sklearn.linear_model import ElasticNetCV

elastic_final = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.9], cv=5, random_state=42)
elastic_final.fit(X_final, y_final)

print("ElasticNet Final R² (Full Data):", elastic_final.score(X_final, y_final))
print("Best alpha:", elastic_final.alpha_)
print("Coefficients:", dict(zip(features, elastic_final.coef_)))


ElasticNet Final R² (Full Data): 0.3372791115569612
Best alpha: 0.014224753082283412
Coefficients: {'RTI_x_Unemployment': np.float64(4.141590381890095e-05), 'Poverty_x_Unemployment': np.float64(-0.0), 'Eviction_Rate_Lag1': np.float64(0.0)}


In [33]:
from sklearn.model_selection import cross_val_score
elastic_scores_final = cross_val_score(elastic_final, X_final, y_final, cv=5)
print("ElasticNet Final Cross-Validated R²:", elastic_scores_final.mean())


ElasticNet Final Cross-Validated R²: -91.02517035729369
